In [13]:
import h3
from h3 import Polygon
import geopandas as gpd
from shapely.geometry import Polygon as ShapelyPolygon
import folium
import pandas as pd

def get_hexagons_for_geojson(geojson_path, resolution):
    # Read the GeoJSON file
    gdf = gpd.read_file(geojson_path)

    # Dictionary to track which polygon(s) each hexagon belongs to
    hexagon_to_cuenca = {}

    # Process each polygon in the GeoJSON
    for idx, row in gdf.iterrows():
        geom = row.geometry
        cuenca_name = row['CUENCA']

        # Handle different geometry types
        if geom.geom_type == 'MultiPolygon':
            polygons = list(geom.geoms)
        else:
            polygons = [geom]

        # Process each polygon
        for polygon in polygons:
            if hasattr(polygon, 'exterior'):
                # Get coordinates and handle 3D coordinates if present
                if len(polygon.exterior.coords[0]) > 2:
                    coords = [(x, y) for x, y, *_ in polygon.exterior.coords]
                else:
                    coords = list(polygon.exterior.coords)

                # Convert to (lat, lng) format for h3
                vertices = [(y, x) for x, y in coords]

                # Create h3 polygon and get cells
                h3_polygon = Polygon(vertices)
                hexagons = h3.polygon_to_cells(h3_polygon, resolution)

                # Associate each hexagon with the current CUENCA
                for hex_id in hexagons:
                    if hex_id in hexagon_to_cuenca:
                        if cuenca_name not in hexagon_to_cuenca[hex_id]:
                            hexagon_to_cuenca[hex_id].append(cuenca_name)
                    else:
                        hexagon_to_cuenca[hex_id] = [cuenca_name]

    # Get all unique hexagons
    all_hexagons = list(hexagon_to_cuenca.keys())

    # Create geometries for each hexagon
    geometries = [ShapelyPolygon([(lng, lat) for lat, lng in h3.cell_to_boundary(h)]) for h in all_hexagons]

    # Create DataFrame with hexagon indices and their CUENCA values
    df_data = {
        'h3_index': all_hexagons,
        'CUENCA': [';'.join(hexagon_to_cuenca[h]) for h in all_hexagons]
    }

    # Create GeoDataFrame
    hexagon_gdf = gpd.GeoDataFrame(
        data=df_data,
        geometry=geometries,
        crs="EPSG:4326"
    )

    return hexagon_gdf

# Path to your GeoJSON file
geojson_path = "/Users/santi/DataspellProjects/Lithium-Jakob/Data/selected_basins.geojson"

# Create GeoDataFrame with hexagons
resolution = 6  # You can adjust this based on your needs
hexagon_gdf = get_hexagons_for_geojson(geojson_path, resolution)

print(f"Created {len(hexagon_gdf)} hexagons at resolution {resolution}")
print(f"Hexagons by CUENCA: {hexagon_gdf['CUENCA'].value_counts().to_dict()}")
hexagon_gdf.head()

# Optional: Save hexagons to a new GeoJSON file
output_path = "/Users/santi/DataspellProjects/Lithium-Jakob/Data/lithium_hexagons_res6.geojson"
hexagon_gdf.to_file(output_path, driver="GeoJSON")

# Optional: Visualize the hexagons colored by CUENCA
def visualize_hexagons_by_cuenca(original_gdf, hexagon_gdf):
    # Create a map centered on the original data
    center = original_gdf.unary_union.centroid
    m = folium.Map(location=[center.y, center.x], zoom_start=8)

    # Add the original polygons
    folium.GeoJson(
        original_gdf,
        name='Original Basins',
        style_function=lambda x: {
            'fillColor': 'blue',
            'color': 'blue',
            'weight': 2,
            'fillOpacity': 0.1
        },
        tooltip=folium.GeoJsonTooltip(fields=['CUENCA'], aliases=['Basin:'])
    ).add_to(m)

    # Get unique CUENCA values for coloring
    unique_cuencas = set()
    for cuenca_str in hexagon_gdf['CUENCA']:
        for cuenca in cuenca_str.split(';'):
            unique_cuencas.add(cuenca)

    # Create color map
    import random
    color_map = {cuenca: f"#{random.randint(0, 0xFFFFFF):06x}" for cuenca in unique_cuencas}

    # Function to determine color based on CUENCA value
    def get_color(cuenca_str):
        cuencas = cuenca_str.split(';')
        if len(cuencas) == 1:
            return color_map[cuencas[0]]
        else:
            # For hexagons in multiple basins, use a distinct color
            return 'purple'

    # Add the hexagons colored by CUENCA
    for idx, row in hexagon_gdf.iterrows():
        folium.GeoJson(
            row.geometry.__geo_interface__,
            style_function=lambda x, cuenca=row['CUENCA']: {
                'fillColor': get_color(cuenca),
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.5
            },
            tooltip=row['CUENCA']
        ).add_to(m)

    # Add a legend
    from branca.element import Template, MacroElement

    template = """
    {% macro html(this, kwargs) %}
    <div style="position: fixed; bottom: 50px; left: 50px; width: 150px; height: auto; z-index:9999; font-size:14px; background-color: white; padding: 10px; border: 2px solid grey;">
    <h4>Legend</h4>
    {% for cuenca in cuencas %}
    <div>
      <span style="background-color: {{colors[cuenca]}}; display: inline-block; width: 12px; height: 12px;"></span>
      <span>{{cuenca}}</span>
    </div>
    {% endfor %}
    <div>
      <span style="background-color: purple; display: inline-block; width: 12px; height: 12px;"></span>
      <span>Multiple basins</span>
    </div>
    </div>
    {% endmacro %}
    """

    macro = MacroElement()
    macro._template = Template(template)
    macro.cuencas = list(unique_cuencas)
    macro.colors = color_map
    #m.get_root().add_child(macro)

    # Add layer control
    #folium.LayerControl().add_to(m)

    return m

# Visualize
original_gdf = gpd.read_file(geojson_path)
m = visualize_hexagons_by_cuenca(original_gdf, hexagon_gdf)
m.save("/Users/santi/DataspellProjects/Lithium-Jakob/Data/lithium_hexagons_visualization.html")
m

Created 640 hexagons at resolution 6
Hexagons by CUENCA: {'Laguna de Guayatayoc': 341, 'Salinas Grandes de Jujuy y Salta': 138, 'Salar de Olaroz': 89, 'Salar de Cauchari': 72}


In [14]:
hexagon_gdf[hexagon_gdf["h3_index"]]

,h3_index,CUENCA,geometry
0,86b3506b7ffffff,Salar de Cauchari,"POLYGON ((-66.87866 -23.99008, -66.90726 -24.0..."
1,86b350627ffffff,Salar de Cauchari,"POLYGON ((-66.66966 -23.90124, -66.69817 -23.9..."
2,86b3507a7ffffff,Salar de Cauchari,"POLYGON ((-66.80663 -23.82564, -66.83519 -23.8..."
3,86b35042fffffff,Salar de Cauchari,"POLYGON ((-66.89567 -23.63998, -66.92424 -23.6..."
4,86b3539afffffff,Salar de Cauchari,"POLYGON ((-66.50526 -24.17251, -66.53374 -24.1..."
...,...,...,...
635,86b35149fffffff,Salinas Grandes de Jujuy y Salta,"POLYGON ((-66.17039 -24.31013, -66.19876 -24.3..."
636,86b35568fffffff,Salinas Grandes de Jujuy y Salta,"POLYGON ((-65.77841 -23.53271, -65.80656 -23.5..."
637,86b35022fffffff,Salinas Grandes de Jujuy y Salta,"POLYGON ((-66.20200 -23.86440, -66.23034 -23.8..."
638,86b351db7ffffff,Salinas Grandes de Jujuy y Salta,"POLYGON ((-66.00858 -23.67702, -66.03684 -23.6..."
